# GET NEXT LINE
- Can use read, malloc, and free

### Notes
- Currently doesnt work with changing buffer size.
- Leaks memory out the ass...
- Does not conform at all with the norm.
- Thoughts:
	- Should look at rearranging make_ret so that the beginning searches the buffer from offset to next \n
	- I didnt make the function with this in mind, but in most cases the buffer will contain many possible strings and should be the first thing searched.
	- right now I have the idea of the buffer is going to be smaller than the line so get many chunks. But I think assuming a large buffer would be better. Like this I can design it so it searches within it's self first. Like all good philosophers should.

In [ ]:
check size_t and ssize_t with read!!!!!!!!!!!!

## get_next_line.h

In [173]:
%%file ../gnl_working/get_next_line.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.h                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:29 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:32 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef GET_NEXT_LINE_H
# define GET_NEXT_LINE_H
# ifndef BUFFER_SIZE
#  define BUFFER_SIZE 1024
# endif
# include <stdlib.h>
# include <unistd.h>
# include <stdint.h>

char	*get_next_line(int fd);
void	*ft_calloc_gnl(size_t num_elem, size_t elem_size);
int		make_ret_gnl(int fd, char **ret, char *buf, size_t buf_size);

#endif

Overwriting ../gnl_working/get_next_line.h


## ~M~A~N~D~A~T~O~R~Y~

### get_next_line.c

In [186]:
%%file ../gnl_working/get_next_line.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:18 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:23 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

char	*get_next_line(int fd)
{
	char			*buf;
	char			*ret;
	static int		fd_table[1024];
	size_t			buf_size;

	buf_size = 1024;
	if (buf_size > BUFFER_SIZE)
		buf_size = BUFFER_SIZE;
	ret = 0;
	buf = (char *)ft_calloc_gnl((buf_size + 1), sizeof(char));
	if (!buf)
		return (0);
	fd_table[fd] = make_ret_gnl(fd, &ret, buf, buf_size);
	free(buf);
	if (fd_table[fd] == -1)
		return (0);
	if (!ret)
		return (0);
	return (ret);
}


Overwriting ../gnl_working/get_next_line.c


### get_next_line_utils.c

In [232]:
%%file ../gnl_working/get_next_line_utils.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   get_next_line_utils.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2022/12/01 13:53:38 by dpentlan          #+#    #+#             */
/*   Updated: 2022/12/01 13:53:41 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "get_next_line.h"

void	*ft_calloc_gnl(size_t num_elem, size_t elem_size)
{
	void	*ptr;
	size_t	i;

	if (num_elem == 0 || elem_size == 0)
	{
		num_elem = 1;
		elem_size = 1;
	}
	if (num_elem >= SIZE_MAX || elem_size >= SIZE_MAX)
		return (0);
	ptr = malloc(num_elem * elem_size);
	if (ptr)
	{
		i = 0;
		while (i < (num_elem * elem_size))
		{
			*(char *)(ptr + i) = 0;
			i++;
		}
	}
	return (ptr);
}

static size_t	ft_strlcat_gnl(char *dest, const char *src, size_t size)
{
	size_t	dest_len;
	size_t	src_len;
	size_t	i;

	dest_len = 0U;
	src_len = 0U;
	i = 0U;
	if (src == 0 || dest == 0)
		return (0);
	while (dest[dest_len] != '\0')
		dest_len++;
	while (src[src_len] != '\0')
		src_len++;
	if (size == 0)
		return (src_len);
	while (src[i] != '\0' && i < size)
	{
		dest[dest_len + i] = src[i];
		i++;
	}
	dest[dest_len + i] = '\0';
	if (dest_len > size)
		return (src_len + size);
	return (dest_len + src_len);
}

static char	*ft_strdup_gnl(const char *src, ssize_t r_size)
{
	char		*str_ret;
	size_t		src_len;
	size_t		i;
	size_t		read_size;

	read_size = (size_t)r_size;
	i = 0;
	src_len = 0;
	if (!src)
	{
		str_ret = (char *)ft_calloc_gnl(read_size + 1, sizeof(char));
		if (!str_ret)
			return (0);
		return (str_ret);
	}
	while (src[src_len])
		src_len++;
	str_ret = (char *)ft_calloc_gnl((src_len + read_size + 1), sizeof(char));
	if (!str_ret)
		return (0);
	i = 0;
	while (src[i])
	{
		str_ret[i] = src[i];
		i++;
	}
	str_ret[i] = 0;
	return (str_ret);
}

static size_t	fill_buffer_gnl(int fd, char *buf, size_t buf_size)
{
	size_t	i;
	ssize_t	r_size;

	i = 0;
	while (i < buf_size)
	{
		r_size = read(fd, &buf[i], 1);
		if (r_size == -1)
			return (-1);
		if (r_size == 0)
			return (i);
		if (buf[i] == '\n')
		{
			buf[i + 1] = 0;
			return (i + 1);
		}
		i++;
	}
	return (i);
}

int	make_ret_gnl(int fd, char **ret, char *buf, size_t buf_size)
{
	ssize_t	r_size;
	char	*ret_2;

	r_size = buf_size;
	while (r_size == (ssize_t)buf_size)
	{
		r_size = fill_buffer_gnl(fd, buf, buf_size);
		if (r_size == -1)
			return (-1);
		if (r_size > 0)
		{
			ret_2 = ft_strdup_gnl(*ret, r_size);
			if (!ret_2)
			{
				free(*ret);
				*ret = 0;
				return (-1);
			}
			free(*ret);
			*ret = ret_2;
			if (!ft_strlcat_gnl(*ret, buf, (size_t)r_size + 1))
				return (-1);
		}
	}
	return (0);
}

Overwriting ../gnl_working/get_next_line_utils.c


### gnl_test

In [233]:
%%file gnl_test.c
#include "../gnl_working/get_next_line.h"
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>

int main (void)
{
	char	*string;
	int		i;
	int		fd1;
//	int		fd2;

	fd1 = open("../gnl_working/tripouille/files/multiple_nlx5", O_RDONLY);
//	fd2 = open("sample.txt", O_RDONLY);
	i = 0;
	while (i < 25)
	{
		string = get_next_line(fd1);
		if (string)
			printf("%s", string);
		free(string);
//		string = get_next_line(fd2);
//		if (string)
//			printf("%s\n", string);
//		free(string);
		i++;
	}
	close(fd1);
//		close(fd2);

	return (0);
}

Overwriting gnl_test.c


In [234]:
%%bash
SIZE1=10000000
SIZE2=1
#rm output*
#cc -Wall -Wextra -Werror -D BUFFER_SIZE=42 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -o gnl_test
gcc -Wall -Wextra -Werror -D BUFFER_SIZE=$SIZE1 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -g -o gnl_test
./gnl_test > output1
cat -vET output1
echo ""
cat -vET ../gnl_working/tripouille/files/41_with_nl
# cat output1
#gcc -Wall -Wextra -Werror -D BUFFER_SIZE=$SIZE2 gnl_test.c ../gnl_working/get_next_line.c ../gnl_working/get_next_line_utils.c -g -o gnl_test
#./gnl_test > output2
valgrind -q --leak-check=full --track-origins=yes -s ./gnl_test
#diff output1 output2

$
$
$
$
$

0123456789012345678901234567890123456789$
0






../gnl_working/get_next_line_utils.c: In function ‘make_ret_gnl’:
../gnl_working/get_next_line_utils.c:132:28: error: comparison of integer expressions of different signedness: ‘size_t’ {aka ‘long unsigned int’} and ‘int’ [-Werror=sign-compare]
  132 |                 if (r_size == -1)
      |                            ^~
cc1: all warnings being treated as errors
==43944== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


## ~B~O~N~U~S~

- Add bonus material here

## Makefile

### Makefile

In [14]:
%%file ../gnl_testing/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2022/12/01 14:36:56 by dpentlan          #+#    #+#              #
#    Updated: 2022/12/01 14:36:58 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

# Not needed for get next line. But in case I want to use this later it's here.
# Note that is not working yet..

NAME = gnl

SRCS =	get_next_line_utils.c \
	get_next_line.c

BONUS_SRCS =

OBJS = ${SRCS:.c=.o}

BONUS_OBJS = ${BONUS_SRCS:.c=.o}

INCS = get_next_line.h

CFLAGS = -Wall -Wextra -Werror

CC = cc

SLIB = ar rcs

all: $(NAME)

$(NAME): $(OBJS) $(INCS)
	@$(SLIB) $(NAME) -o $(OBJS)

bonus: $(OBJS) $(BONUS_OBJS) $(INCS)
	@$(SLIB) $(NAME) -o $(OBJS) $(BONUS_OBJS)

%.o: %.c
	@$(CC) ${CFLAGS} -c $< -o ${<:.c=.o}

clean:
	@rm -f $(OBJS) $(BONUS_OBJS)

fclean:	clean
	@rm -f $(NAME)

re:	fclean all


Overwriting ../gnl_testing/Makefile


### Build Mandatory

In [ ]:
%%bash
cd ../gnl_working
# add build commands here

## Scratch

### File descriptors
- I found out after making this that the struct _IO_FILE is included in the standard library and is used to help interaction with files, but it is not used with read, open, write, close, etc. 
- It is used with fread, fopen, fwrite, and fclose.

In [331]:
%%file fd_examine.h
#ifndef FD_EXAMINE_H_
# define FD_EXAMINE_H_
# include <stdio.h>
# include <unistd.h>

void	print_file_struct_properties(char *label, FILE *p_file);

#endif

Overwriting fd_examine.h


In [332]:
%%file fd_examine.c
#include "fd_examine.h"

void	print_file_struct_properties(char *label, FILE *p_file)
{
	printf("label,size,member type,member name,member description,value,offset from beginning of struct\n");
	printf("whole struct,%ld,,,\n", sizeof(*p_file));
	printf("%s,%d,int,_flags,no description,%d,%ld\n", label, 8, stdin->_flags, (((unsigned long int) &(stdin->_flags)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_ptr,Current read pointer,%p,%ld\n", label, 8, stdin->_IO_read_ptr, (((unsigned long int) &(stdin->_IO_read_ptr)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_end,End of get area,%p,%ld\n", label, 8, stdin->_IO_read_end, (((unsigned long int) &(stdin->_IO_read_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_read_base,Start of putback+get area,%p,%ld\n", label, 8, stdin->_IO_read_base, (((unsigned long int) &(stdin->_IO_read_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_base,Start of put area,%p,%ld\n", label, 8, stdin->_IO_write_base, (((unsigned long int) &(stdin->_IO_write_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_ptr,Current put pointer,%p,%ld\n", label, 8, stdin->_IO_write_ptr, (((unsigned long int) &(stdin->_IO_write_ptr)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_write_end,End of put area,%p,%ld\n", label, 8, stdin->_IO_write_end, (((unsigned long int) &(stdin->_IO_write_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_buf_base,Start of reserve area,%p,%ld\n", label, 8, stdin->_IO_buf_base, (((unsigned long int) &(stdin->_IO_buf_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_buf_end,End of reserve area,%p,%ld\n", label, 8, stdin->_IO_buf_end, (((unsigned long int) &(stdin->_IO_buf_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_save_base,Pointer to start of non-current get area,%p,%ld\n", label, 8, stdin->_IO_save_base, (((unsigned long int) &(stdin->_IO_save_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_backup_base,Pointer to first valid character of backup area,%p,%ld\n", label, 8, stdin->_IO_backup_base, (((unsigned long int) &(stdin->_IO_backup_base)) - ((unsigned long int) stdin)));
	printf("%s,%d,char *,_IO_save_end,Pointer to end of non-current get area,%p,%ld\n", label, 8, stdin->_IO_save_end, (((unsigned long int) &(stdin->_IO_save_end)) - ((unsigned long int) stdin)));
	printf("%s,%d,struct _IO_marker *,_markers,no description,%p,%ld\n", label, 8, stdin->_markers, (((unsigned long int) &(stdin->_markers)) - ((unsigned long int) stdin)));
	printf("%s,%d,struct _IO_FILE *,_chain,no description,%p,%ld\n", label, 8, stdin->_chain, (((unsigned long int) &(stdin->_chain)) - ((unsigned long int) stdin)));
	printf("%s,%d,int,_fileno,no description,%d,%ld\n", label, 4, stdin->_fileno, (((unsigned long int) &(stdin->_fileno)) - ((unsigned long int) stdin)));
	//printf("%s,%d,int,_blksize,no description,%d,%ld\n", label, 4, stdin->_blksize, (((unsigned long int) &(stdin->_blksize)) - ((unsigned long int) stdin)));
	printf("%s,%d,int,_flags2,no description,%d,%ld\n", label, 4, stdin->_flags2, (((unsigned long int) &(stdin->_flags2)) - ((unsigned long int) stdin)));
	printf("%s,%d,_IO_off_t,_old_offset,This used to be _offset but it's too small,%ld,%ld\n", label, 8, stdin->_old_offset, (((unsigned long int) &(stdin->_old_offset)) - ((unsigned long int) stdin)));
	printf("%s,%d,unsigned short,_cur_column,no description,%d,%ld\n", label, 2, stdin->_cur_column, (((unsigned long int) &(stdin->_cur_column)) - ((unsigned long int) stdin)));
	printf("%s,%d,signed char,_vtable_offset,no description,%d,%ld\n", label, 8, stdin->_vtable_offset, (((unsigned long int) &(stdin->_vtable_offset)) - ((unsigned long int) stdin)));
	printf("%s,%d,char,_shortbuf,no description,%s,%ld\n", label, 8, stdin->_shortbuf, (((unsigned long int) &(stdin->_shortbuf)) - ((unsigned long int) stdin)));
	printf("%s,%d,_IO_lock_t *,_lock,no description,%p,%ld\n", label, 8, stdin->_lock, (((unsigned long int) &(stdin->_lock)) - ((unsigned long int) stdin)));

	return ;
}

Overwriting fd_examine.c


In [333]:
%%file fd_test.c
#include <stdio.h>
#include <unistd.h>
#include "fd_examine.h"

int main (void)
{
	//printf("i/o    :descriptor\tpointer\t\t\n");
	//printf("stdin  :%d\t\t%p\n", STDIN_FILENO, stdin);
	//printf("stdout :%d\t\t%p\n", STDOUT_FILENO, stdout);
	// printf("stderr :%d\t\t%p\n", STDERR_FILENO, stderr);

	print_file_struct_properties("stdin", stdin);
	//print_file_struct_properties("stdout", stdout);
	//print_file_struct_properties("stderr", stderr);

	return (0);
}

Overwriting fd_test.c


In [334]:
%%bash
gcc fd_test.c fd_examine.c -g
./a.out > output.csv

### Stack Frame
- This is a simple program for me to use with gdb to view the stack frame.

In [24]:
%%file stack_frame.c
#include <stdlib.h>

char	*return_string(void)
{
	char	*str;
	str = (char *)malloc(10);
	return (str);
}

int foo (int n)
{
	static char *here;
	here = 0;
	here = return_string();
	return (n + 1);
}

int main (void)
{
	int	n;

	n = 0;
	n = foo(n);
	return (0);
}

Overwriting stack_frame.c


In [25]:
%%bash
gcc stack_frame.c -g -o stack_frame
./stack_frame
valgrind --leak-check=full --show-leak-kinds=all -s ./stack_frame

==62578== Memcheck, a memory error detector
==62578== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==62578== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==62578== Command: ./stack_frame
==62578== 
==62578== 
==62578== HEAP SUMMARY:
==62578==     in use at exit: 10 bytes in 1 blocks
==62578==   total heap usage: 1 allocs, 0 frees, 10 bytes allocated
==62578== 
==62578== 10 bytes in 1 blocks are still reachable in loss record 1 of 1
==62578==    at 0x4848899: malloc (in /usr/libexec/valgrind/vgpreload_memcheck-amd64-linux.so)
==62578==    by 0x10915E: return_string (stack_frame.c:6)
==62578==    by 0x109187: foo (stack_frame.c:14)
==62578==    by 0x1091B3: main (stack_frame.c:23)
==62578== 
==62578== LEAK SUMMARY:
==62578==    definitely lost: 0 bytes in 0 blocks
==62578==    indirectly lost: 0 bytes in 0 blocks
==62578==      possibly lost: 0 bytes in 0 blocks
==62578==    still reachable: 10 bytes in 1 blocks
==62578==         suppressed: 0 byt